In [1]:
from dotenv import dotenv_values

config = dotenv_values("../.env")

import json
import os

import requests
from openai import OpenAI

In [2]:
base_url = "https://api.tokenfactory.nebius.com/v1/chat/completions/"
API_KEY = config["NEBIUS_API_KEY"]
model = "Qwen/Qwen3-30B-A3B-Instruct-2507"

# openai and smolagents

In [17]:
client = OpenAI(base_url=base_url, api_key=API_KEY)

In [19]:
from huggingface_hub import InferenceClient

In [20]:
client = InferenceClient(provider="nebius", api_key=API_KEY, model=model)

In [21]:
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Tell me an easy ice cream recipe."},
]

In [22]:
completion = client.chat.completions.create(messages=messages, max_tokens=150)

Model Qwen/Qwen3-235B-A22B-Thinking-2507 is in staging mode for provider nebius. Meant for test purposes only.


In [27]:
completion.choices[0].message.reasoning_content

"Okay, the user asked for an easy ice cream recipe. Let me start by understanding what they really need. They probably want something simple with minimal ingredients and no fancy equipment. Maybe they're a beginner or just looking for a quick treat without much hassle.\n\nFirst, I should consider common pain points. Traditional ice cream recipes often need an ice cream maker, which not everyone has. So, avoiding that is key. Also, ingredients should be staples most people have at home. Heavy cream and sweetened condensed milk are good because they're common in many kitchens for desserts.\n\nI recall a 3-ingredient no-churn method. It's popular because it's straightforward. Heavy cream whipped to stiff peaks, mixed with condensed milk and vanilla. That covers"

In [ ]:
from smolagents import ToolCallingAgent, Tool, InferenceClientModel, tool


In [35]:
@tool
def add(a: int | float, b: int | float) -> int | float:
    """This function adds two numbers and returns their sum

    Args:
        a (int | float): The first number
        b (int | float): The second number

    Returns:
        int | float: The sum of the two numbers
    """
    return a + b

In [44]:
add.to_dict()

{'name': 'add',
 'code': 'from smolagents import Tool\nfrom typing import Any, Optional\n\nclass SimpleTool(Tool):\n    name = "add"\n    description = "This function adds two numbers and returns their sum"\n    inputs = {\'a\': {\'type\': [\'integer\', \'number\'], \'description\': \'The first number\'}, \'b\': {\'type\': [\'integer\', \'number\'], \'description\': \'The second number\'}}\n    output_type = "any"\n\n    def forward(self, a: int | float, b: int | float) -> int | float:\n        """This function forwards two numbers and returns their sum\n\n        Args:\n            a (int | float): The first number\n            b (int | float): The second number\n\n        Returns:\n            int | float: The sum of the two numbers\n        """\n        return a + b',
 'requirements': ['smolagents']}

In [36]:
print(add.to_code_prompt())

def add(a: ['integer', 'number'], b: ['integer', 'number']) -> any:
    """This function adds two numbers and returns their sum

    Args:
        a: The first number
        b: The second number
    """


In [29]:
model = InferenceClientModel(model_id=model, provider="nebius", api_key=API_KEY)

In [45]:
agent = ToolCallingAgent(tools=[add], model=model)
agent.run("What is 1 + 343234?")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ What is 1 + 343234?                                                                                             │
│                                                                                                                 │
╰─ InferenceClientModel - Qwen/Qwen3-235B-A22B-Thinking-2507 ─────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Model Qwen/Qwen3-235B-A22B-Thinking-2507 is in staging mode for provider nebius. Meant for test purposes only.


╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'add' with arguments: {'a': 1, 'b': 343234}                                                       │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Observations: 343235

...

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'final_answer' with arguments: {'answer': '343235'}                                               │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Observations: 343235

Final answer: 343235

[Step 2: Duration 5.56 seconds| Input tokens: 2,394 | Output tokens: 498]

'343235'

# llamaindex

In [3]:
from llama_index.llms.huggingface_api import HuggingFaceInferenceAPI

In [4]:
llm = HuggingFaceInferenceAPI(
    model_name=model, temperature=0.7, max_tokens=100, provider="nebius"
)

In [5]:
response = llm.complete("Hello, how are you?")
print(response)

Hello! I'm functioning well, thank you for asking! I'm here and ready to help you with whatever you need. How can I assist you today? 😊


In [6]:
from llama_index.llms.huggingface_api import HuggingFaceInferenceAPI
from llama_index.core.agent.workflow import AgentWorkflow, ToolCallResult, AgentStream


In [7]:
def add(a: int, b: int) -> int:
    """Add two numbers"""
    return a + b


def subtract(a: int, b: int) -> int:
    """Subtract two numbers"""
    return a - b


def multiply(a: int, b: int) -> int:
    """Multiply two numbers"""
    return a * b


def divide(a: int, b: int) -> int:
    """Divide two numbers"""
    return a / b

In [8]:
agent = AgentWorkflow.from_tools_or_functions(
    tools_or_functions=[subtract, multiply, divide, add],
    llm=llm,
    system_prompt="You are a math agent that can add, subtract, multiply, and divide numbers using provided tools.",
)

In [9]:
handler = agent.run("What is (2 + 2) * 2?")
async for ev in handler.stream_events():
    if isinstance(ev, ToolCallResult):
        print("")
        print("Called tool: ", ev.tool_name, ev.tool_kwargs, "=>", ev.tool_output)
    elif isinstance(ev, AgentStream):  # showing the thought process
        print(ev.delta, end="", flush=True)

resp = await handler
resp

Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: add
Action Input: {"a": 2, "b": 2}
Observation: 4
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: multiply
Action Input: {"a": 4, "b": 2}
Observation: 8
Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: 8
Called tool:  add {'a': 2, 'b': 2} => 4
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: multiply
Action Input: {'a': 4, 'b': 2}
Called tool:  multiply {'a': 4, 'b': 2} => 8
Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: 8

AgentOutput(response=ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, additional_kwargs={}, blocks=[TextBlock(block_type='text', text='8')]), structured_response=None, current_agent_name='Agent', raw=ChatCompletionStreamOutput(choices=[ChatCompletionStreamOutputChoice(delta=ChatCompletionStreamOutputDelta(role=None, content='8', reasoning=None, tool_call_id=None, tool_calls=None), index=0, finish_reason=None, logprobs=None, token_ids=None)], created=1763355366, id='chatcmpl-3b30677add9744bda6c7776e53c5f0cf', model='Qwen/Qwen3-30B-A3B-Instruct-2507', system_fingerprint=None, usage=None, object='chat.completion.chunk'), tool_calls=[ToolCallResult(tool_name='add', tool_kwargs={'a': 2, 'b': 2}, tool_id='721050ea-f110-45f0-8283-70b7b4d54058', tool_output=ToolOutput(blocks=[TextBlock(block_type='text', text='4')], tool_name='add', raw_input={'args': (), 'kwargs': {'a': 2, 'b': 2}}, raw_output=4, is_error=False), return_direct=False), ToolCallResult(tool_name='multiply', tool_kwargs={'a

In [10]:
from llama_index.core.workflow import Context

ctx = Context(agent)

response = await agent.run("My name is Bob.", ctx=ctx)
response = await agent.run("What was my name again?", ctx=ctx)
response

AgentOutput(response=ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, additional_kwargs={}, blocks=[TextBlock(block_type='text', text='Your name is Bob.')]), structured_response=None, current_agent_name='Agent', raw=ChatCompletionStreamOutput(choices=[ChatCompletionStreamOutputChoice(delta=ChatCompletionStreamOutputDelta(role=None, content='.', reasoning=None, tool_call_id=None, tool_calls=None), index=0, finish_reason=None, logprobs=None, token_ids=None)], created=1763355433, id='chatcmpl-d92eb7b9a52649d5998a496847ff4952', model='Qwen/Qwen3-30B-A3B-Instruct-2507', system_fingerprint=None, usage=None, object='chat.completion.chunk'), tool_calls=[ToolCallResult(tool_name='add', tool_kwargs={'a': 5, 'b': 3}, tool_id='14c87c49-415f-446a-8b00-c47fed1a2fce', tool_output=ToolOutput(blocks=[TextBlock(block_type='text', text='8')], tool_name='add', raw_input={'args': (), 'kwargs': {'a': 5, 'b': 3}}, raw_output=8, is_error=False), return_direct=False)], retry_messages=[])